# Agents Basics with MCP

In [2]:
from agents import Agent, Runner, WebSearchTool

agent = Agent(
    name="Assistant",
    model="gpt-5",
    tools=[
        WebSearchTool(),
    ],
)


result = await Runner.run(agent, "What is the model context protocol and how to build a simple agent with it in super simpler terms?")
print(result.final_output)

Super simple version first

- What MCP is: Think of it like USB‑C for AI. It’s a standard “plug” that lets an AI app (your agent) safely talk to outside tools and data without bespoke wiring each time. You write small MCP servers that expose capabilities, and any MCP‑aware agent can discover and use them. ([modelcontextprotocol.info](https://modelcontextprotocol.info/en/docs/introduction/?utm_source=chatgpt.com))
- The pieces:
  - MCP server: a tiny program that offers tools (functions), resources (read‑only data), and optional prompts to the model. ([modelcontextprotocol.io](https://modelcontextprotocol.io/quickstart?utm_source=chatgpt.com))
  - MCP host/client (your agent or an app like Claude Desktop): connects to one or more servers, lists their tools, and lets the model call them with your approval. ([modelcontextprotocol.info](https://modelcontextprotocol.info/en/docs/introduction/?utm_source=chatgpt.com))
  - Transports: clients and servers talk over simple channels like stdio o

In [3]:
import json

from typing_extensions import TypedDict, Any

from agents import Agent, FunctionTool, RunContextWrapper, function_tool


class Location(TypedDict):
    lat: float
    long: float

@function_tool  
async def fetch_weather(location: Location) -> str:
    
    """Fetch the weather for a given location.

    Args:
        location: The location to fetch the weather for.
    """
    # In real life, we'd fetch the weather from a weather API
    return f"The weather in this location: {location.lat}, {location.long} is sunny"


@function_tool(name_override="fetch_data")  
def read_file(ctx: RunContextWrapper[Any], path: str, directory: str | None = None) -> str:
    """Read the contents of a file.

    Args:
        path: The path to the file to read.
        directory: The directory to read the file from.
    """
    # In real life, we'd read the file from the file system
    return "<file contents>"


agent = Agent(
    name="Assistant",
    tools=[fetch_weather, read_file],  
)

for tool in agent.tools:
    if isinstance(tool, FunctionTool):
        print(tool.name)
        print(tool.description)
        print(json.dumps(tool.params_json_schema, indent=2))
        print()

fetch_weather
Fetch the weather for a given location.
{
  "$defs": {
    "Location": {
      "properties": {
        "lat": {
          "title": "Lat",
          "type": "number"
        },
        "long": {
          "title": "Long",
          "type": "number"
        }
      },
      "required": [
        "lat",
        "long"
      ],
      "title": "Location",
      "type": "object",
      "additionalProperties": false
    }
  },
  "properties": {
    "location": {
      "description": "The location to fetch the weather for.",
      "properties": {
        "lat": {
          "title": "Lat",
          "type": "number"
        },
        "long": {
          "title": "Long",
          "type": "number"
        }
      },
      "required": [
        "lat",
        "long"
      ],
      "title": "Location",
      "type": "object",
      "additionalProperties": false
    }
  },
  "required": [
    "location"
  ],
  "title": "fetch_weather_args",
  "type": "object",
  "additionalProperties